In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
%matplotlib inline

In [2]:
dataset = pd.read_csv('./Diabetes_Data_1999_2008.csv')
admission_src = pd.read_csv('./Admission_Source_Mapping.csv')
admission_type = pd.read_csv('./Admission_Type_Mapping.csv')
discharge_disposition = pd.read_csv('./Discharge_Disposition_Mapping.csv')

In [4]:
cleaned_df = dataset[dataset.readmitted == "NO"] #only looking at first encounters
cleaned_df = cleaned_df[cleaned_df.discharge_disposition_id != {18, 19, 20}]
cleaned_df = cleaned_df.iloc[:, :19]
numMed = cleaned_df.iloc[:, 14:15]
demographics_df = cleaned_df.iloc[:, 2:5]
diag1_df = cleaned_df.iloc[:, 18:19]
cleaned_df = pd.concat([numMed, diag1_df, demographics_df], axis = 1, join = 'inner')
cleaned_df = cleaned_df[cleaned_df.race != "?"]
cleaned_df = cleaned_df[cleaned_df.gender != "Unknown/Invalid"]
diag_1Map = {}
diag_1List = cleaned_df["diag_1"].value_counts().index.tolist()[:50]

diag_1Count = cleaned_df["diag_1"].value_counts().values.tolist()[:50]

for i, v in enumerate(diag_1List):
    diag_1Map[v] = i
for i in cleaned_df.diag_1:
    if i not in diag_1Map:
        diag_1Map[i] = 50

cleaned_df["diag_1"] = cleaned_df["diag_1"].map(diag_1Map)

print(cleaned_df)

        num_medications  diag_1             race  gender      age
0                     1      50        Caucasian  Female   [0-10)
2                    13      50  AfricanAmerican  Female  [20-30)
3                    16      45        Caucasian    Male  [30-40)
4                     8      50        Caucasian    Male  [40-50)
6                    21       0        Caucasian    Male  [60-70)
...                 ...     ...              ...     ...      ...
101759               15      21        Caucasian    Male  [80-90)
101762               18      26  AfricanAmerican  Female  [80-90)
101763                9      10        Caucasian    Male  [70-80)
101764               21      11        Caucasian  Female  [80-90)
101765                3      37        Caucasian    Male  [70-80)

[53316 rows x 5 columns]


In [6]:
cleaneddf.to_csv("cleaned_data.csv")

NameError: name 'df' is not defined